# Temporal Out-of-Sample Validation Tutorial

In this tutorial, we'll demonstrate how to perform temporal out-of-sample validation using the full CAMELS-Australia dataset instead of sample data. We'll leverage the built-in tools from the `hydroml` package to automatically handle data loading and execute the training and evaluation pipeline.

The workflow will:
1. Load the CAMELS-Australia dataset
2. Set up temporal splits for calibration and validation
3. Train and fine-tune a model
4. Evaluate model performance

Let's get started!


In [1]:
from hydroml.config.config import Config
from hydroml.workflow.evaluation import train_finetune_evaluate

# Set up the path mapping

As this training is going to load the dataset automativally from camels australia, we need to set up the path mapping first. The method to set up the path mapping is the same as the one in the 03_build_a_path_mapping.ipynb tutorial.


In [2]:
# We assume that the path mapping is already set up with the platform name 'win_2'.
platform='win_2'

We also need a list of basins for calibration and validation. These catchments should be already available in the camels australia postprocessed dataset and awara postprocessed dataset. So the pipeline will automatically load the data.

In [3]:
basins_file = '../sample_data/basins.txt'
with open(basins_file, 'r') as f:
    catchment_ids = f.read().splitlines()

Using these information we can set up the config object.

For a full explanation of the config object please refer to the readme documentation.

In [4]:
# we only use 2 catchments for calibration and validation for this tutorial.
config = Config(cal={'periods' : [['1991-01-01', '2014-01-01']], 'catchment_ids':catchment_ids[:2] },  
                val={'periods' : [['1985-01-01', '1990-01-01']], 'catchment_ids':catchment_ids[:2] }, 
                name = 'evaluation_tutorial',
                lstm_hidden_size=4, # we are selecting a small lstm size for this tutorial.
                device='cpu',
                platform=platform, # to introduce the paths stored in config/path_mapping/win.yaml
                max_epochs=2, # and we reduce the number of epochs to 2 for this tutorial.
                )    

# we  need to call this function to set up the version name - or you can manually set it up.
config.set_new_version_name()


Now we can run the **training and evaluation pipeline**. This pipeline works as follows:

1. Trains a **continental model** and evaluates it on the validation set.  
2. Starts the **fine-tuning process**, where the continental model is fine-tuned for each catchment and then evaluated on that specific catchment.

The results will be stored in the following structure:

| **File/Folder**                          | **Description**                                                      | **Path**                                                          |
|------------------------------------------|----------------------------------------------------------------------|-------------------------------------------------------------------|
| **Model Weights**                        | Weights of the trained continental model.                           | `root_path/VERSION_NAME/last.ckpt`                                |
| **Config File**                          | Configuration settings for the pipeline.                            | `root_path/VERSION_NAME/config.yaml`                              |
| **Transformer Parameters**               | Parameters for the transformer model.                               | `root_path/VERSION_NAME/params.yaml`                              |
| **Simulation Results (Continental Model)** | Simulation output for the continental model.                        | `root_path/VERSION_NAME/simulation.nc`                            |
| **Metrics**                              | Evaluation metrics for the continental model.                       | `root_path/VERSION_NAME/metrics.nc`                               |
| **Fine-tuned Models**                    | Fine-tuned models for each catchment.                               | `root_path/VERSION_NAME/finetune_all/catchment_id/VERSION_NAME/last.ckpt` |

- Note 1: that the finetuned models are stored with a same structure as the continental model.
- Note 2: the finetuned models do not have a params.yaml file as they are config to use the transformer parameter in the continental model.


In [5]:
train_finetune_evaluate(config)

Transforming data: calculating transform parameters and saving to \\fs1-cbr.nexus.csiro.au\{d61-coastal-forecasting-wp3}\work\sho108_handover\models\evaluation_tutorial\241218102129_2bd1\params.yaml


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-vTEL7SXK-py3.11\Lib\site-packages\torch\utils\data\dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


valid data points per catchment {0: 5381, 1: 2219}


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-vTEL7SXK-py3.11\Lib\site-packages\torch\utils\data\dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
GPU available: False, used: False


Transforming data: loading transform parameters from \\fs1-cbr.nexus.csiro.au\{d61-coastal-forecasting-wp3}\work\sho108_handover\models\evaluation_tutorial\241218102129_2bd1\params.yaml
\\fs1-cbr.nexus.csiro.au\{d61-coastal-forecasting-wp3}\work\sho108_handover\models\evaluation_tutorial\241218102129_2bd1


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=15` reached.
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-vTEL7SXK-py3.11\Lib\site-packages\torch\utils\data\dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Transforming data: loading transform parameters from \\fs1-cbr.nexus.csiro.au\{d61-coastal-forecasting-wp3}\work\sho108_handover\models\evaluation_tutorial\241218102129_2bd1\params.yaml


WindowsPath('//fs1-cbr.nexus.csiro.au/{d61-coastal-forecasting-wp3}/work/sho108_handover/models/evaluation_tutorial/241218102129_2bd1')

So the new set of results for the simulation over the validtion period using the continental model is stored in the following path:


In [6]:
import xarray as xr

p = config.current_path / config.version /'results' / 'simulation.nc'
xr.open_dataset(p)


<xarray.Dataset> Size: 35kB
Dimensions:       (date: 1462, catchment_id: 2, lead_time: 1, feature: 1)
Coordinates:
  * date          (date) datetime64[ns] 12kB 1986-01-01 ... 1990-01-01
  * catchment_id  (catchment_id) <U7 56B '912101A' '912105A'
Dimensions without coordinates: lead_time, feature
Data variables:
    prediction    (lead_time, feature, date, catchment_id) float32 12kB ...
    y             (lead_time, feature, date, catchment_id) float32 12kB ...